In [6]:
import sympy as sp
from sympy import init_printing
init_printing()
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent.parent))
from cores.dynamical_systems.quadrotor_2d import Quadrotor2D
import torch
import numpy as np

In [26]:
mass = 2.0
inertia = 0.2
arm_length = 0.4
gravity = 9.81

In [27]:
system_torch = Quadrotor2D(mass, inertia, arm_length, torch.float32)

In [28]:
x, y, theta, x_dot, y_dot, theta_dot = sp.symbols(r'x y \theta \dot{x} \dot{y} \dot{\theta}')
u1, u2 = sp.symbols(r'u_1, u_2')
states = sp.Matrix([x, y, theta, x_dot, y_dot, theta_dot])
controls = sp.Matrix([u1, u2])

In [29]:
f1 = x_dot
f2 = y_dot
f3 = theta_dot
sin_theta = sp.sin(theta)
cos_theta = sp.cos(theta)
f4 = -(u1 + u2) * sin_theta / mass - gravity * sin_theta
f5 = (u1 + u2) * cos_theta / mass + gravity * (cos_theta - 1)
f6 = arm_length * (- u1 + u2) / inertia
f = sp.Matrix([f1, f2, f3, f4, f5, f6])

# Test f(x,u)

In [30]:
f_func = sp.lambdify((states, controls), f)
state_np = np.array([1., 2., 3., 4., 5., 6.], dtype=np.float32)
control_np = np.array([7., 8.], dtype=np.float32)
print(f_func(state_np, control_np))


[[  4.       ]
 [  5.       ]
 [  6.       ]
 [ -2.4427872]
 [-26.946774 ]
 [  2.       ]]


In [31]:
state_torch = torch.tensor(state_np, dtype=torch.float32).unsqueeze(0)
control_torch = torch.tensor(control_np, dtype=torch.float32).unsqueeze(0)
f_torch = system_torch(state_torch, control_torch)
print(f_torch)

tensor([[  4.0000,   5.0000,   6.0000,  -2.4428, -26.9468,   2.0000]])


# Test df/dx

In [32]:
df_dx = f.jacobian(states)
df_dx_func = sp.lambdify((states, controls), df_dx)
print(df_dx_func(state_np, control_np))

[[ 0.          0.          0.          1.          0.          0.        ]
 [ 0.          0.          0.          0.          1.          0.        ]
 [ 0.          0.          0.          0.          0.          1.        ]
 [ 0.          0.         17.13677216  0.          0.          0.        ]
 [ 0.          0.         -2.44278717  0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.        ]]


In [34]:
df_dx_torch = system_torch.f_dx(state_torch, control_torch)
print(df_dx_torch)

tensor([[[ 0.0000,  0.0000,  0.0000,  1.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000,  1.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  1.0000],
         [ 0.0000,  0.0000, 17.1368,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000, -2.4428,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000]]])


# Test df/du

In [35]:
df_du = f.jacobian(controls)
df_du_func = sp.lambdify((states, controls), df_du)
print(df_du_func(state_np, control_np))

[[ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [-0.07056    -0.07056   ]
 [-0.49499625 -0.49499625]
 [-2.          2.        ]]


In [36]:
df_du_torch = system_torch.f_du(state_torch, control_torch)
print(df_du_torch)

tensor([[[ 0.0000,  0.0000],
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000],
         [-0.0706, -0.0706],
         [-0.4950, -0.4950],
         [-2.0000,  2.0000]]])


# d^2f/dx^2

In [40]:
f4_dxdx = f4.diff(states).jacobian(states)
print(f4_dxdx)

Matrix([[0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, -0.5*(-u_1 - u_2)*sin(\theta) + 9.81*sin(\theta), 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]])


In [42]:
f5_dxdx = f5.diff(states).jacobian(states)
print(f5_dxdx)

Matrix([[0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, -0.5*(u_1 + u_2)*cos(\theta) - 9.81*cos(\theta), 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]])
